In [1]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime

In [2]:
# se descargan los archivos a dataframe
credits= pd.read_csv(r"F:\HENRY\DATA SCIENCE\PI\PI 1\credits.csv")
movies= pd.read_csv(r"F:\HENRY\DATA SCIENCE\PI\PI 1\movies_dataset.csv")

C:\Users\Samuel\AppData\Local\Temp\ipykernel_8548\3760531119.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies= pd.read_csv(r"F:\HENRY\DATA SCIENCE\PI\PI 1\movies_dataset.csv")


In [3]:
# Se seleccionan  las columnas a utilizar
movies= movies[['belongs_to_collection', 'budget', 'genres', 'id', 'original_language', 'overview',
       'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 
       'vote_average', 'vote_count']]
movies.head(1)


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0


# SE REALIZA ETL PARA LAS COLUMNAS ANIDADAS

In [4]:
# SE REALIZA ETL PARA LAS COLUMNAS ANIDADAS

# funcion que extraer los valores de la clave 'name' 
import ast

def obtener_nombre_diccionario(diccionario):
    if isinstance(diccionario, str):#verifica si el los valores del dicionario son str
        try:
            diccionario = ast.literal_eval(diccionario)# convierte a un diccionario valido
            if isinstance(diccionario, dict):# verifica si es un dicionario
                return diccionario.get('name','')# si hay name en el diccionario te da el valor y si no existe name de devuelve vacio 
        except (SyntaxError, ValueError):#si hay un error lo pasa y sigue con la otra fila 
            pass
    return ''

# Aplicar la función a la columna 'belongs_to_collection' y almacenar los resultados en una nueva columna 'belongs_to_collection_names'
movies['belongs_to_collection_names'] = movies['belongs_to_collection'].fillna('').apply(obtener_nombre_diccionario) # se rellenan los nulos a '' y se aplica la funcion
 

In [5]:
# Generar una funcion para extraer los datos de listas de diccionarios

def obtener_nombre_lista_dicc(lista_diccionarios):
    nombres = []
    if isinstance(lista_diccionarios, list):# se evalua si los datos son una lista
        for diccionario in lista_diccionarios: # se recorre la lista 
            if isinstance(diccionario, dict): # se evalua si los elementos son diccionarios
                nombres.append(diccionario.get('name', ''))# si lo son se extraen el valor de la clave name y se adiciona a la lista nombres
    elif isinstance(lista_diccionarios, str):# si es un str 
        try:
            lista_diccionarios = ast.literal_eval(lista_diccionarios)# los datos se convierten a diccionarios validos
            if isinstance(lista_diccionarios, list):# se evalua si los datos son una lista
                for diccionario in lista_diccionarios:# se recorre la lista 
                    if isinstance(diccionario, dict):# se evalua si los elementos son diccionarios
                        nombres.append(diccionario.get('name', ''))# si lo son se extraen el valor de la clave name y se adiciona a la lista nombres
        except (SyntaxError, ValueError):
            pass
    return ','.join(nombres)# junta los datos de la lista mediante ","

# Se aplica la funcion para la columan genres
movies['genres_names'] = movies['genres'].apply(obtener_nombre_lista_dicc)


In [6]:
# Se aplica la funcion para la columan production_companies
movies['production_companies_names'] = movies['production_companies'].apply(obtener_nombre_lista_dicc)


In [7]:
# Se aplica la funcion para la columan production countries
movies['production_countries_names'] = movies['production_countries'].apply(obtener_nombre_lista_dicc)


In [8]:
# Se aplica la funcion para la columan spoken_languages
movies['spoken_languages_names'] = movies['spoken_languages'].apply(obtener_nombre_lista_dicc)


# Se rellenan los nulos por 0 y convertirlos a float

In [9]:
# Rellenar los campos 
def convertir_columna_float(columna):
    columna = pd.to_numeric(columna, errors='coerce')  # Convertir a float y rellenar no convertibles con 0
    return columna          
                        
movies["budget"] = movies["budget"].fillna(0).apply(convertir_columna_float).fillna(0)

movies["revenue"]=movies["revenue"].fillna(0)
movies[["budget","revenue"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   budget   45466 non-null  float64
 1   revenue  45466 non-null  float64
dtypes: float64(2)
memory usage: 710.5 KB


# Creando la columna return 

In [10]:
movies["return"]=movies["revenue"]/movies["budget"]
movies["return"]=movies["return"].fillna(0)


# Borrando los valores nulos de release_date y creando la columna release_year

In [11]:
movies = movies.dropna(subset=['release_date'])
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies['release_year'] = movies['release_date'].dt.year

# DESARROLANDO EL ETL DE CREDITO

In [12]:
credits= pd.read_csv(r"F:\HENRY\DATA SCIENCE\PI\PI 1\credits.csv")
credits.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [13]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
credits['id'] = pd.to_numeric(credits['id'], errors='coerce')


In [14]:
x="name"
def extraer_names(lista):
    nombre=[]
    if isinstance(lista,list):
        for diccionario in lista:
            nombre.append(diccionario.get(x,'')) 
    elif isinstance(lista,str):
        try:
            lista=ast.literal_eval(lista)
            if isinstance(lista,list):
                for diccionario in lista:
                    if isinstance(diccionario,dict):
                        nombre.append(diccionario.get(x,''))
        except (SyntaxError,ValueError):
            pass
    return ",".join(nombre)



In [15]:
credits["names_cast"]=credits["cast"].apply(extraer_names)
credits["names_cast"]


0        Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal...
1        Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra...
2        Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ...
3        Whitney Houston,Angela Bassett,Loretta Devine,...
4        Steve Martin,Diane Keaton,Martin Short,Kimberl...
                               ...                        
45471              Leila Hatami,Kourosh Tahami,Elham Korda
45472    Angel Aquino,Perry Dizon,Hazel Orencio,Joel To...
45473    Erika Eleniak,Adam Baldwin,Julie du Page,James...
45474    Iwan Mosschuchin,Nathalie Lissenko,Pavel Pavlo...
45475                                                     
Name: names_cast, Length: 45476, dtype: object

In [16]:
x="character"
def extraer_names(lista):
    nombre=[]
    if isinstance(lista,list):
        for diccionario in lista:
            nombre.append(diccionario.get(x,'')) 
    elif isinstance(lista,str):
        try:
            lista=ast.literal_eval(lista)
            if isinstance(lista,list):
                for diccionario in lista:
                    if isinstance(diccionario,dict):
                        nombre.append(diccionario.get(x,''))
        except (SyntaxError,ValueError):
            pass
    return ",".join(nombre)

In [17]:
credits["character_cast"]=credits["cast"].apply(extraer_names)
credits["character_cast"]


0        Woody (voice),Buzz Lightyear (voice),Mr. Potat...
1        Alan Parrish,Samuel Alan Parrish / Van Pelt,Ju...
2        Max Goldman,John Gustafson,Ariel Gustafson,Mar...
3        Savannah 'Vannah' Jackson,Bernadine 'Bernie' H...
4        George Banks,Nina Banks,Franck Eggelhoffer,Ann...
                               ...                        
45471                                                   ,,
45472    Sister Angela,Homer,Crazy Woman/Virgin,Amang T...
45473    Emily Shaw,Det. Mark Winston,Jayne Ferré,Alex ...
45474                                                 ,,,,
45475                                                     
Name: character_cast, Length: 45476, dtype: object

In [18]:
x="department"
def extraer_names(lista):
    nombre=[]
    if isinstance(lista,list):
        for diccionario in lista:
            nombre.append(diccionario.get(x,'')) 
    elif isinstance(lista,str):
        try:
            lista=ast.literal_eval(lista)
            if isinstance(lista,list):
                for diccionario in lista:
                    if isinstance(diccionario,dict):
                        nombre.append(diccionario.get(x,''))
        except (SyntaxError,ValueError):
            pass
    return ",".join(nombre)

credits["department_crew"]=credits["crew"].apply(extraer_names)
credits["department_crew"]

0        Directing,Writing,Writing,Writing,Writing,Prod...
1        Production,Writing,Sound,Directing,Editing,Pro...
2                           Directing,Writing,Writing,Crew
3        Directing,Writing,Production,Production,Produc...
4        Sound,Camera,Writing,Production,Writing,Direct...
                               ...                        
45471    Directing,Production,Camera,Crew,Editing,Sound...
45472             Directing,Writing,Art,Sound,Editing,Crew
45473               Directing,Writing,Writing,Sound,Camera
45474                                 Directing,Production
45475                                            Directing
Name: department_crew, Length: 45476, dtype: object

In [19]:
x="job"
def extraer_names(lista):
    nombre=[]
    if isinstance(lista,list):
        for diccionario in lista:
            nombre.append(diccionario.get(x,'')) 
    elif isinstance(lista,str):
        try:
            lista=ast.literal_eval(lista)
            if isinstance(lista,list):
                for diccionario in lista:
                    if isinstance(diccionario,dict):
                        nombre.append(diccionario.get(x,''))
        except (SyntaxError,ValueError):
            pass
    return ",".join(nombre)

credits["job_crew"]=credits["crew"].apply(extraer_names)
credits["job_crew"]

0        Director,Screenplay,Screenplay,Screenplay,Scre...
1        Executive Producer,Screenplay,Original Music C...
2               Director,Characters,Writer,Sound Recordist
3        Director,Screenplay,Producer,Producer,Producer...
4        Original Music Composer,Director of Photograph...
                               ...                        
45471    Director,Producer,Camera Supervisor,Script,Edi...
45472    Director,Writer,Production Design,Music,Editor...
45473    Director,Screenplay,Screenplay,Original Music ...
45474                                    Director,Producer
45475                                             Director
Name: job_crew, Length: 45476, dtype: object

In [20]:
x="name"
def extraer_names(lista):
    nombre=[]
    if isinstance(lista,list):
        for diccionario in lista:
            nombre.append(diccionario.get(x,'')) 
    elif isinstance(lista,str):
        try:
            lista=ast.literal_eval(lista)
            if isinstance(lista,list):
                for diccionario in lista:
                    if isinstance(diccionario,dict):
                        nombre.append(diccionario.get(x,''))
        except (SyntaxError,ValueError):
            pass
    return ",".join(nombre)

credits["name_crew"]=credits["crew"].apply(extraer_names)
credits["name_crew"]

0        John Lasseter,Joss Whedon,Andrew Stanton,Joel ...
1        Larry J. Franco,Jonathan Hensleigh,James Horne...
2        Howard Deutch,Mark Steven Johnson,Mark Steven ...
3        Forest Whitaker,Ronald Bass,Ronald Bass,Ezra S...
4        Alan Silvestri,Elliot Davis,Nancy Meyers,Nancy...
                               ...                        
45471    Hamid Nematollah,Hamid Nematollah,Farshad Moha...
45472    Lav Diaz,Lav Diaz,Dante Perez,Lav Diaz,Lav Dia...
45473    Mark L. Lester,C. Courtney Joyner,Jeffrey Gold...
45474                 Yakov Protazanov,Joseph N. Ermolieff
45475                                        Daisy Asquith
Name: name_crew, Length: 45476, dtype: object

In [21]:
credits=credits[['id', 'character_cast','names_cast', 'department_crew', 'job_crew',
       'name_crew']]
credits

,id,character_cast,names_cast,department_crew,job_crew,name_crew
0,862,"Woody (voice),Buzz Lightyear (voice),Mr. Potat...","Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal...","Directing,Writing,Writing,Writing,Writing,Prod...","Director,Screenplay,Screenplay,Screenplay,Scre...","John Lasseter,Joss Whedon,Andrew Stanton,Joel ..."
1,8844,"Alan Parrish,Samuel Alan Parrish / Van Pelt,Ju...","Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra...","Production,Writing,Sound,Directing,Editing,Pro...","Executive Producer,Screenplay,Original Music C...","Larry J. Franco,Jonathan Hensleigh,James Horne..."
2,15602,"Max Goldman,John Gustafson,Ariel Gustafson,Mar...","Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ...","Directing,Writing,Writing,Crew","Director,Characters,Writer,Sound Recordist","Howard Deutch,Mark Steven Johnson,Mark Steven ..."
3,31357,"Savannah 'Vannah' Jackson,Bernadine 'Bernie' H...","Whitney Houston,Angela Bassett,Loretta Devine,...","Directing,Writing,Production,Production,Produc...","Director,Screenplay,Producer,Producer,Producer...","Forest Whitaker,Ronald Bass,Ronald Bass,Ezra S..."
4,11862,"George Banks,Nina Banks,Franck Eggelhoffer,Ann...","Steve Martin,Diane Keaton,Martin Short,Kimberl...","Sound,Camera,Writing,Production,Writing,Direct...","Original Music Composer,Director of Photograph...","Alan Silvestri,Elliot Davis,Nancy Meyers,Nancy..."
...,...,...,...,...,...,...
45471,439050,",,","Leila Hatami,Kourosh Tahami,Elham Korda","Directing,Production,Camera,Crew,Editing,Sound...","Director,Producer,Camera Supervisor,Script,Edi...","Hamid Nematollah,Hamid Nematollah,Farshad Moha..."
45472,111109,"Sister Angela,Homer,Crazy Woman/Virgin,Amang T...","Angel Aquino,Perry Dizon,Hazel Orencio,Joel To...","Directing,Writing,Art,Sound,Editing,Crew","Director,Writer,Production Design,Music,Editor...","Lav Diaz,Lav Diaz,Dante Perez,Lav Diaz,Lav Dia..."
45473,67758,"Emily Shaw,Det. Mark Winston,Jayne Ferré,Alex ...","Erika Eleniak,Adam Baldwin,Julie du Page,James...","Directing,Writing,Writing,Sound,Camera","Director,Screenplay,Screenplay,Original Music ...","Mark L. Lester,C. Courtney Joyner,Jeffrey Gold..."
45474,227506,",,,,","Iwan Mosschuchin,Nathalie Lissenko,Pavel Pavlo...","Directing,Production","Director,Producer","Yakov Protazanov,Joseph N. Ermolieff"


In [22]:
movies=movies[['id','budget','popularity','release_date','revenue',
       'status','title','vote_average','vote_count',
       'belongs_to_collection_names', 'genres_names',
       'production_companies_names', 'production_countries_names',
       'spoken_languages_names', 'return', 'release_year']]
movies


,id,budget,popularity,release_date,revenue,status,title,vote_average,vote_count,belongs_to_collection_names,genres_names,production_companies_names,production_countries_names,spoken_languages_names,return,release_year
0,862.0,30000000.0,21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0,Toy Story Collection,"Animation,Comedy,Family",Pixar Animation Studios,United States of America,English,12.451801,1995.0
1,8844.0,65000000.0,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413.0,,"Adventure,Fantasy,Family","TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,"English,Français",4.043035,1995.0
2,15602.0,0.0,11.7129,1995-12-22,0.0,Released,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"Romance,Comedy","Warner Bros.,Lancaster Gate",United States of America,English,0.000000,1995.0
3,31357.0,16000000.0,3.859495,1995-12-22,81452156.0,Released,Waiting to Exhale,6.1,34.0,,"Comedy,Drama,Romance",Twentieth Century Fox Film Corporation,United States of America,English,5.090760,1995.0
4,11862.0,0.0,8.387519,1995-02-10,76578911.0,Released,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,Comedy,"Sandollar Productions,Touchstone Pictures",United States of America,English,inf,1995.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,30840.0,0.0,5.683753,1991-05-13,0.0,Released,Robin Hood,5.7,26.0,,"Drama,Action,Romance","Westdeutscher Rundfunk (WDR),Working Title Fil...","Canada,Germany,United Kingdom,United States of...",English,0.000000,1991.0
45462,111109.0,0.0,0.178241,2011-11-17,0.0,Released,Century of Birthing,9.0,3.0,,Drama,Sine Olivia,Philippines,,0.000000,2011.0
45463,67758.0,0.0,0.903007,2003-08-01,0.0,Released,Betrayal,3.8,6.0,,"Action,Drama,Thriller",American World Pictures,United States of America,English,0.000000,2003.0
45464,227506.0,0.0,0.003503,1917-10-21,0.0,Released,Satan Triumphant,0.0,0.0,,,Yermoliev,Russia,,0.000000,1917.0


In [23]:
credits=credits[['id', 'character_cast','names_cast', 'department_crew', 'job_crew', 'name_crew']]
credits

,id,character_cast,names_cast,department_crew,job_crew,name_crew
0,862,"Woody (voice),Buzz Lightyear (voice),Mr. Potat...","Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal...","Directing,Writing,Writing,Writing,Writing,Prod...","Director,Screenplay,Screenplay,Screenplay,Scre...","John Lasseter,Joss Whedon,Andrew Stanton,Joel ..."
1,8844,"Alan Parrish,Samuel Alan Parrish / Van Pelt,Ju...","Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra...","Production,Writing,Sound,Directing,Editing,Pro...","Executive Producer,Screenplay,Original Music C...","Larry J. Franco,Jonathan Hensleigh,James Horne..."
2,15602,"Max Goldman,John Gustafson,Ariel Gustafson,Mar...","Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ...","Directing,Writing,Writing,Crew","Director,Characters,Writer,Sound Recordist","Howard Deutch,Mark Steven Johnson,Mark Steven ..."
3,31357,"Savannah 'Vannah' Jackson,Bernadine 'Bernie' H...","Whitney Houston,Angela Bassett,Loretta Devine,...","Directing,Writing,Production,Production,Produc...","Director,Screenplay,Producer,Producer,Producer...","Forest Whitaker,Ronald Bass,Ronald Bass,Ezra S..."
4,11862,"George Banks,Nina Banks,Franck Eggelhoffer,Ann...","Steve Martin,Diane Keaton,Martin Short,Kimberl...","Sound,Camera,Writing,Production,Writing,Direct...","Original Music Composer,Director of Photograph...","Alan Silvestri,Elliot Davis,Nancy Meyers,Nancy..."
...,...,...,...,...,...,...
45471,439050,",,","Leila Hatami,Kourosh Tahami,Elham Korda","Directing,Production,Camera,Crew,Editing,Sound...","Director,Producer,Camera Supervisor,Script,Edi...","Hamid Nematollah,Hamid Nematollah,Farshad Moha..."
45472,111109,"Sister Angela,Homer,Crazy Woman/Virgin,Amang T...","Angel Aquino,Perry Dizon,Hazel Orencio,Joel To...","Directing,Writing,Art,Sound,Editing,Crew","Director,Writer,Production Design,Music,Editor...","Lav Diaz,Lav Diaz,Dante Perez,Lav Diaz,Lav Dia..."
45473,67758,"Emily Shaw,Det. Mark Winston,Jayne Ferré,Alex ...","Erika Eleniak,Adam Baldwin,Julie du Page,James...","Directing,Writing,Writing,Sound,Camera","Director,Screenplay,Screenplay,Original Music ...","Mark L. Lester,C. Courtney Joyner,Jeffrey Gold..."
45474,227506,",,,,","Iwan Mosschuchin,Nathalie Lissenko,Pavel Pavlo...","Directing,Production","Director,Producer","Yakov Protazanov,Joseph N. Ermolieff"


In [24]:
movies_credits=pd.merge(movies,credits,on="id")
movies_credits["id"]=movies_credits["id"].astype(int)
movies_credits.head(2)


,id,budget,popularity,release_date,revenue,status,title,vote_average,vote_count,belongs_to_collection_names,...,production_companies_names,production_countries_names,spoken_languages_names,return,release_year,character_cast,names_cast,department_crew,job_crew,name_crew
0,862,30000000.0,21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0,Toy Story Collection,...,Pixar Animation Studios,United States of America,English,12.451801,1995.0,"Woody (voice),Buzz Lightyear (voice),Mr. Potat...","Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal...","Directing,Writing,Writing,Writing,Writing,Prod...","Director,Screenplay,Screenplay,Screenplay,Scre...","John Lasseter,Joss Whedon,Andrew Stanton,Joel ..."
1,8844,65000000.0,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413.0,,...,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,"English,Français",4.043035,1995.0,"Alan Parrish,Samuel Alan Parrish / Van Pelt,Ju...","Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra...","Production,Writing,Sound,Directing,Editing,Pro...","Executive Producer,Screenplay,Original Music C...","Larry J. Franco,Jonathan Hensleigh,James Horne..."


In [25]:
movies_credits['release_date'] = pd.to_datetime(movies_credits['release_date'])

# Crear una nueva columna para almacenar los nombres de los meses
movies_credits['release_month'] = ''

# Iterar sobre cada fila del DataFrame
for index, row in movies_credits.iterrows():
    release_date = row['release_date']
    release_month = release_date.strftime('%B')  # Obtener el nombre del mes en formato completo
    movies_credits.at[index, 'release_month'] = release_month
import locale

# Establecer la configuración regional en español
locale.setlocale(locale.LC_TIME, 'es_ES')

movies_credits['release_month'] = movies_credits['release_date'].dt.strftime('%B').str.lower().apply(lambda x: x.capitalize())

# Mostrar el DataFrame actualizado
movies_credits

,id,budget,popularity,release_date,revenue,status,title,vote_average,vote_count,belongs_to_collection_names,...,production_countries_names,spoken_languages_names,return,release_year,character_cast,names_cast,department_crew,job_crew,name_crew,release_month
0,862,30000000.0,21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0,Toy Story Collection,...,United States of America,English,12.451801,1995.0,"Woody (voice),Buzz Lightyear (voice),Mr. Potat...","Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal...","Directing,Writing,Writing,Writing,Writing,Prod...","Director,Screenplay,Screenplay,Screenplay,Scre...","John Lasseter,Joss Whedon,Andrew Stanton,Joel ...",Octubre
1,8844,65000000.0,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413.0,,...,United States of America,"English,Français",4.043035,1995.0,"Alan Parrish,Samuel Alan Parrish / Van Pelt,Ju...","Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra...","Production,Writing,Sound,Directing,Editing,Pro...","Executive Producer,Screenplay,Original Music C...","Larry J. Franco,Jonathan Hensleigh,James Horne...",Diciembre
2,15602,0.0,11.7129,1995-12-22,0.0,Released,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,...,United States of America,English,0.000000,1995.0,"Max Goldman,John Gustafson,Ariel Gustafson,Mar...","Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ...","Directing,Writing,Writing,Crew","Director,Characters,Writer,Sound Recordist","Howard Deutch,Mark Steven Johnson,Mark Steven ...",Diciembre
3,31357,16000000.0,3.859495,1995-12-22,81452156.0,Released,Waiting to Exhale,6.1,34.0,,...,United States of America,English,5.090760,1995.0,"Savannah 'Vannah' Jackson,Bernadine 'Bernie' H...","Whitney Houston,Angela Bassett,Loretta Devine,...","Directing,Writing,Production,Production,Produc...","Director,Screenplay,Producer,Producer,Producer...","Forest Whitaker,Ronald Bass,Ronald Bass,Ezra S...",Diciembre
4,11862,0.0,8.387519,1995-02-10,76578911.0,Released,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,...,United States of America,English,inf,1995.0,"George Banks,Nina Banks,Franck Eggelhoffer,Ann...","Steve Martin,Diane Keaton,Martin Short,Kimberl...","Sound,Camera,Writing,Production,Writing,Direct...","Original Music Composer,Director of Photograph...","Alan Silvestri,Elliot Davis,Nancy Meyers,Nancy...",Febrero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45446,30840,0.0,5.683753,1991-05-13,0.0,Released,Robin Hood,5.7,26.0,,...,"Canada,Germany,United Kingdom,United States of...",English,0.000000,1991.0,"Sir Robert Hode,Maid Marian,Little John,Sir Mi...","Patrick Bergin,Uma Thurman,David Morrissey,Jür...","Directing,Writing,Writing,Writing,Production,S...","Director,Writer,Writer,Story,Producer,Music,Di...","John Irvin,Sam Resnick,John McGrath,Sam Resnic...",Mayo
45447,111109,0.0,0.178241,2011-11-17,0.0,Released,Century of Birthing,9.0,3.0,,...,Philippines,,0.000000,2011.0,"Sister Angela,Homer,Crazy Woman/Virgin,Amang T...","Angel Aquino,Perry Dizon,Hazel Orencio,Joel To...","Directing,Writing,Art,Sound,Editing,Crew","Director,Writer,Production Design,Music,Editor...","Lav Diaz,Lav Diaz,Dante Perez,Lav Diaz,Lav Dia...",Noviembre
45448,67758,0.0,0.903007,2003-08-01,0.0,Released,Betrayal,3.8,6.0,,...,United States of America,English,0.000000,2003.0,"Emily Shaw,Det. Mark Winston,Jayne Ferré,Alex ...","Erika Eleniak,Adam Baldwin,Julie du Page,James...","Directing,Writing,Writing,Sound,Camera","Director,Screenplay,Screenplay,Original Music ...","Mark L. Lester,C. Courtney Joyner,Jeffrey Gold...",Agosto
45449,227506,0.0,0.003503,1917-10-21,0.0,Released,Satan Triumphant,0.0,0.0,,...,Russia,,0.000000,1917.0,",,,,","Iwan Mosschuchin,Nathalie Lissenko,Pavel Pavlo...","Directing,Production","Director,Producer","Yakov Protazanov,Joseph N. Ermolieff",Octubre


In [27]:
movies_credits.to_csv(r"F:\HENRY\DATA SCIENCE\PI\PI 1\movies_credits.csv",index=False)